<a href="https://colab.research.google.com/github/dinarsadykow/cv_hw/blob/main/Stepik_SQL_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install/Load Lib
SQL stepik https://stepik.org/course/63054


In [2]:
%%capture

# ------------------------------------------------------
# pySpark Tutorial from https://towardsdatascience.com/pyspark-on-google-colab-101-d31830b238be

!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pickle5
!pip install pyspark_dist_explore

# ------------------------------------------------------
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 4 --executor-memory 4g --driver-memory 3g pyspark-shell'

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, LongType

# ------------------------------------------------------
spark = (
      SparkSession.builder\
        .master("local")\
        .appName("sdr")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.maxResultSize", "4g")\
        #.config("spark.sql.crossJoin.enabled", "true")\
        .config("spark.sql.catalogImplementation","hive")
        .config("spark.sql.hive.convertMetastoreParquet","false")
        .getOrCreate()
      )

import pyspark.sql.functions as F
import pyspark.sql.types as T

# ------------------------------------------------------
spark

In [3]:
def pDF_to_sDF(
      df_input,
      table_name = 'table1',
      mySchema = None
  ):
  df_pandas = df_input.copy()
  df_sp = spark \
        .createDataFrame(df_pandas , schema=mySchema ) 
  df_sp.createOrReplaceTempView(table_name)
  df_sp.printSchema()
  df_sp.show()
  return df_sp

# Data Load/Send to Spark: SQL Table

## 1.6 Таблица "Командировки", запросы на выборку

In [4]:
import pandas as pd

t = """('Баранов П.Е.','Москва',700 , '2020-01-12', '2020-01-17'),
('Абрамова К.А.','Владивосток',450 , '2020-01-14', '2020-01-27'),
('Семенов И.В.','Москва',700 , '2020-01-23', '2020-01-31'),
('Ильиных Г.Р.','Владивосток', 450, '2020-01-12', '2020-02-02'),
('Колесов С.П.','Москва',700 , '2020-02-01', '2020-02-06'),
('Баранов П.Е.','Москва', 700, '2020-02-14', '2020-02-22'),
('Абрамова К.А.','Москва', 700, '2020-02-23', '2020-03-01'),
('Лебедев Т.К.','Москва', 700, '2020-03-03', '2020-03-06'),
('Колесов С.П.','Новосибирск',450 , '2020-02-27', '2020-03-12'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-03-29', '2020-04-05'),
('Абрамова К.А.','Москва',700 , '2020-04-06', '2020-04-14'),
('Баранов П.Е.','Новосибирск',450 , '2020-04-18', '2020-05-04'),
('Лебедев Т.К.','Томск',450 , '2020-05-20', '2020-05-31'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-06-01', '2020-06-03'),
('Абрамова К.А.','Санкт-Петербург', 700, '2020-05-28', '2020-06-04'),
('Федорова А.Ю.','Новосибирск',450 , '2020-05-25', '2020-06-04'),
('Колесов С.П.','Новосибирск', 450, '2020-06-03', '2020-06-12'),
('Федорова А.Ю.','Томск', 450, '2020-06-20', '2020-06-26'),
('Абрамова К.А.','Владивосток', 450, '2020-07-02', '2020-07-13'),
('Баранов П.Е.','Воронеж', 450, '2020-07-19', '2020-07-25')"""

df_pd = pd.DataFrame( [ x \
                .replace(')','') \
                .replace('(','') \
                .replace("'",'') \
                .split(',') for x in t.split('\n')] ).drop(5,axis=1)
df_pd.columns = ['name','city','per_diem','date_first','date_last']
df_pd['per_diem'] = df_pd['per_diem'].astype('int32')

df_pd['date_first'] = pd.to_datetime( df_pd['date_first'].map(lambda x: x.strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='ignore')
df_pd['date_last'] = pd.to_datetime( df_pd['date_last'].map(lambda x: x.strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='ignore')

df_pd

,name,city,per_diem,date_first,date_last
0,Баранов П.Е.,Москва,700,2020-01-12,2020-01-17
1,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27
2,Семенов И.В.,Москва,700,2020-01-23,2020-01-31
3,Ильиных Г.Р.,Владивосток,450,2020-01-12,2020-02-02
4,Колесов С.П.,Москва,700,2020-02-01,2020-02-06
5,Баранов П.Е.,Москва,700,2020-02-14,2020-02-22
6,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
7,Лебедев Т.К.,Москва,700,2020-03-03,2020-03-06
8,Колесов С.П.,Новосибирск,450,2020-02-27,2020-03-12
9,Семенов И.В.,Санкт-Петербург,700,2020-03-29,2020-04-05


In [5]:
mySchema = StructType([
                       T.StructField("name", T.StringType(), True)\
                       ,T.StructField("city", T.StringType(), True)\
                       ,T.StructField("per_diem", T.IntegerType(), True)\
                       ,T.StructField("date_first", T.TimestampType(), True)\
                       ,T.StructField("date_last", T.TimestampType(), True)
                       ])

df_sp = spark \
      .createDataFrame(df_pd ,schema=mySchema ) \
      .withColumn('date_first', F.to_date(F.col('date_first')) ) \
      .withColumn('date_last', F.to_date(F.col('date_last')) )

df_sp.createOrReplaceTempView("trip")

df_sp.printSchema()
df_sp.show()

root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- per_diem: integer (nullable = true)
 |-- date_first: date (nullable = true)
 |-- date_last: date (nullable = true)

+-------------+---------------+--------+----------+----------+
|         name|           city|per_diem|date_first| date_last|
+-------------+---------------+--------+----------+----------+
| Баранов П.Е.|         Москва|     700|2020-01-12|2020-01-17|
|Абрамова К.А.|    Владивосток|     450|2020-01-14|2020-01-27|
| Семенов И.В.|         Москва|     700|2020-01-23|2020-01-31|
| Ильиных Г.Р.|    Владивосток|     450|2020-01-12|2020-02-02|
| Колесов С.П.|         Москва|     700|2020-02-01|2020-02-06|
| Баранов П.Е.|         Москва|     700|2020-02-14|2020-02-22|
|Абрамова К.А.|         Москва|     700|2020-02-23|2020-03-01|
| Лебедев Т.К.|         Москва|     700|2020-03-03|2020-03-06|
| Колесов С.П.|    Новосибирск|     450|2020-02-27|2020-03-12|
| Семенов И.В.|Санкт-Петербург|     700|2020-0

## Практика

###Задание_1
Вывести из таблицы trip информацию о командировках тех сотрудников, фамилия которых заканчивается на букву «а», в отсортированном по убыванию даты последнего дня командировки виде. В результат включить столбцы name, city, per_diem, date_first, date_last.

In [6]:
sql_str = """ select * from trip """
spark.sql(sql_str).toPandas()

,name,city,per_diem,date_first,date_last
0,Баранов П.Е.,Москва,700,2020-01-12,2020-01-17
1,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27
2,Семенов И.В.,Москва,700,2020-01-23,2020-01-31
3,Ильиных Г.Р.,Владивосток,450,2020-01-12,2020-02-02
4,Колесов С.П.,Москва,700,2020-02-01,2020-02-06
5,Баранов П.Е.,Москва,700,2020-02-14,2020-02-22
6,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
7,Лебедев Т.К.,Москва,700,2020-03-03,2020-03-06
8,Колесов С.П.,Новосибирск,450,2020-02-27,2020-03-12
9,Семенов И.В.,Санкт-Петербург,700,2020-03-29,2020-04-05


In [7]:
sql_161 = """SELECT
    name
    , city
    , per_diem
    , date_first
    , date_last

FROM trip

WHERE name LIKE '%а %'
ORDER BY date_last DESC"""

spark.sql(sql_161).toPandas()

,name,city,per_diem,date_first,date_last
0,Абрамова К.А.,Владивосток,450,2020-07-02,2020-07-13
1,Федорова А.Ю.,Томск,450,2020-06-20,2020-06-26
2,Абрамова К.А.,Санкт-Петербург,700,2020-05-28,2020-06-04
3,Федорова А.Ю.,Новосибирск,450,2020-05-25,2020-06-04
4,Абрамова К.А.,Москва,700,2020-04-06,2020-04-14
5,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
6,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27


###Задание_2
Вывести в алфавитном порядке фамилии и инициалы тех сотрудников, которые были в командировке в Москве.

In [8]:
sql_162 = """SELECT
    name

FROM trip

WHERE city = 'Москва'
ORDER BY name ASC"""

spark.sql(sql_162).toPandas()

,name
0,Абрамова К.А.
1,Абрамова К.А.
2,Баранов П.Е.
3,Баранов П.Е.
4,Колесов С.П.
5,Лебедев Т.К.
6,Семенов И.В.


###Задание_3
Для каждого города посчитать, сколько раз сотрудники в нем были.  Информацию вывести в отсортированном в алфавитном порядке по названию городов. Вычисляемый столбец назвать Количество. 

In [9]:
sql_163 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY city ASC"""

spark.sql(sql_163).toPandas()

,city,Kolichestvo
0,Владивосток,3
1,Воронеж,1
2,Москва,7
3,Новосибирск,4
4,Санкт-Петербург,3
5,Томск,2


###Задание_4
Вывести два города, в которых чаще всего были в командировках сотрудники. Вычисляемый столбец назвать Количество.

In [10]:
sql_164 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY Kolichestvo DESC
LIMIT 2"""

spark.sql(sql_164).toPandas()

,city,Kolichestvo
0,Москва,7
1,Новосибирск,4


###Задание_5
Вывести информацию о командировках во все города кроме Москвы и Санкт-Петербурга (фамилии и инициалы сотрудников, город ,  длительность командировки в днях, при этом первый и последний день относится к периоду командировки). Последний столбец назвать Длительность. Информацию вывести в упорядоченном по убыванию длительности поездки, а потом по убыванию названий городов (в обратном алфавитном порядке).

In [11]:
sql_165 = """SELECT
    name
    , city
    , DATEDIFF(date_last,date_first)+1 AS Dlitelnost

FROM trip
WHERE city NOT IN ('Москва','Санкт-Петербург')
ORDER BY Dlitelnost DESC, city DESC"""

spark.sql(sql_165).toPandas()

,name,city,Dlitelnost
0,Ильиных Г.Р.,Владивосток,22
1,Баранов П.Е.,Новосибирск,17
2,Колесов С.П.,Новосибирск,15
3,Абрамова К.А.,Владивосток,14
4,Лебедев Т.К.,Томск,12
5,Абрамова К.А.,Владивосток,12
6,Федорова А.Ю.,Новосибирск,11
7,Колесов С.П.,Новосибирск,10
8,Федорова А.Ю.,Томск,7
9,Баранов П.Е.,Воронеж,7


###Задание_6
Вывести информацию о командировках сотрудника(ов), которые были самыми короткими по времени. В результат включить столбцы name, city, date_first, date_last.

In [12]:
sql_166 = """SELECT

    name, city, date_first, date_last

FROM trip

WHERE (DATEDIFF(date_last,date_first)+1)
            =(
                SELECT MIN(DATEDIFF(date_last,date_first))+1 FROM trip
                )"""

spark.sql(sql_166).toPandas()

,name,city,date_first,date_last
0,Семенов И.В.,Санкт-Петербург,2020-06-01,2020-06-03


###Задание_7
Вывести информацию о командировках, начало и конец которых относятся к одному месяцу (год может быть любой). В результат включить столбцы name, city, date_first, date_last. Строки отсортировать сначала  в алфавитном порядке по названию города, а затем по фамилии сотрудника .

In [13]:
sql_167 = """

SELECT

    name, city, date_first, date_last

FROM trip

WHERE MONTH(date_last)=MONTH(date_first)

ORDER BY city, name

"""

spark.sql(sql_167).toPandas()

,name,city,date_first,date_last
0,Абрамова К.А.,Владивосток,2020-01-14,2020-01-27
1,Абрамова К.А.,Владивосток,2020-07-02,2020-07-13
2,Баранов П.Е.,Воронеж,2020-07-19,2020-07-25
3,Абрамова К.А.,Москва,2020-04-06,2020-04-14
4,Баранов П.Е.,Москва,2020-01-12,2020-01-17
5,Баранов П.Е.,Москва,2020-02-14,2020-02-22
6,Колесов С.П.,Москва,2020-02-01,2020-02-06
7,Лебедев Т.К.,Москва,2020-03-03,2020-03-06
8,Семенов И.В.,Москва,2020-01-23,2020-01-31
9,Колесов С.П.,Новосибирск,2020-06-03,2020-06-12


###Задание_8
Вывести название месяца и количество командировок для каждого месяца. Считаем, что командировка относится к некоторому месяцу, если она началась в этом месяце. Информацию вывести сначала в отсортированном по убыванию количества, а потом в алфавитном порядке по названию месяца виде. Название столбцов – Месяц и Количество.

In [14]:
sql_168 = """

SELECT

    date_format(date_first,'MMMM') AS Month
    , COUNT(*) AS Cnt

FROM trip

GROUP BY date_format(date_first,'MMMM') 

ORDER BY Cnt DESC, Month ASC
;

"""

spark.sql(sql_168).toPandas()

,Month,Cnt
0,February,4
1,January,4
2,June,3
3,May,3
4,April,2
5,July,2
6,March,2


###Задание_9
Вывести сумму суточных (произведение количества дней командировки и размера суточных) для командировок, первый день которых пришелся на февраль или март 2020 года. Значение суточных для каждой командировки занесено в столбец per_diem. Вывести фамилию и инициалы сотрудника, город, первый день командировки и сумму суточных. Последний столбец назвать Сумма. Информацию отсортировать сначала  в алфавитном порядке по фамилиям сотрудников, а затем по убыванию суммы суточных.

In [15]:
sql_169 = """

SELECT

    name
    , city
    , date_first
    , (DATEDIFF(date_last,date_first)*per_diem+per_diem) AS Summa

FROM trip

WHERE MONTH(date_first) IN (2,3)

ORDER BY name ASC, Summa DESC

"""

spark.sql(sql_169).toPandas()

,name,city,date_first,Summa
0,Абрамова К.А.,Москва,2020-02-23,5600
1,Баранов П.Е.,Москва,2020-02-14,6300
2,Колесов С.П.,Новосибирск,2020-02-27,6750
3,Колесов С.П.,Москва,2020-02-01,4200
4,Лебедев Т.К.,Москва,2020-03-03,2800
5,Семенов И.В.,Санкт-Петербург,2020-03-29,5600


###Задание_10
Вывести фамилию с инициалами и общую сумму суточных, полученных за все командировки для тех сотрудников, которые были в командировках больше чем 3 раза, в отсортированном по убыванию сумм суточных виде. Последний столбец назвать Сумма.

Только для этого задания изменена строка таблицы trip:

*4	Ильиных Г.Р.	Владивосток	450	2020-01-12	2020-03-02*




In [16]:
sql_1690 = """

SELECT

    name
    , SUM( (DATEDIFF(date_last,date_first)*per_diem+per_diem)  ) AS Summa

FROM trip

WHERE name IN (SELECT name FROM trip GROUP BY name HAVING COUNT(*)>3 )

GROUP BY name

ORDER BY Summa DESC

"""

spark.sql(sql_1690).toPandas()

,name,Summa
0,Абрамова К.А.,29200
1,Баранов П.Е.,21300


## 1.7 Таблица "Нарушения ПДД", запросы корректировки

In [17]:
t1="""('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости(от 40 до 60)', 500.00, '2020-01-12', '2020-01-17'),
       ('Абрамова К.А.', 'О111АВ', 'Проезд на запрещающий сигнал', 1000.00, '2020-01-14', '2020-02-27'),
       ('Яковлев Г.Р.', 'Т330ТТ', 'Превышение скорости(от 20 до 40)', 500.00, '2020-01-23', '2020-02-23'),
       ('Яковлев Г.Р.', 'М701АА', 'Превышение скорости(от 20 до 40)', NULL, '2020-01-12', NULL),
       ('Колесов С.П.', 'К892АХ', 'Превышение скорости(от 20 до 40)', NULL, '2020-02-01', NULL),
       ('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости(от 40 до 60)', NULL, '2020-02-14 ', NULL),
       ('Абрамова К.А.', 'О111АВ', 'Проезд на запрещающий сигнал', NULL, '2020-02-23', NULL),
       ('Яковлев Г.Р.', 'Т330ТТ', 'Проезд на запрещающий сигнал', NULL, '2020-03-03', NULL)"""

df_pd2 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .replace("'",'') \
                .split(',')] for x in t1.split('\n')] ).drop(6,axis=1)
df_pd2.columns = ['name','number_plate','violation','sum_fine','date_violation','date_payment']

df_pd2['sum_fine'] = pd.to_numeric( df_pd2['sum_fine'], downcast='integer', errors='coerce' )

df_pd2['name'] = df_pd2['name'].map( lambda x: str(x).replace(')','').replace('(','') )

df_pd2['date_violation'] = pd.to_datetime( df_pd2['date_violation'].map(lambda x: str(x).replace(')','').replace('(','').strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='coerce')
df_pd2['date_payment'] = pd.to_datetime( df_pd2['date_payment'].map(lambda x: str(x).replace(')','').replace('(','').strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='coerce')

df_pd2

,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500.0,2020-01-12,2020-01-17
1,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.0,2020-01-14,2020-02-27
2,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500.0,2020-01-23,2020-02-23
3,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),NaN,2020-01-12,NaT
4,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),NaN,2020-02-01,NaT
5,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),NaN,2020-02-14,NaT
6,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,NaN,2020-02-23,NaT
7,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,NaN,2020-03-03,NaT


In [18]:
mySchema = StructType([
                       T.StructField("name", T.StringType(), True)\
                       ,T.StructField("number_plate", T.StringType(), True)\
                       ,T.StructField("violation", T.StringType(), True)\
                       ,T.StructField("sum_fine", T.FloatType(), True)\

                       ,T.StructField("date_violation", T.TimestampType(), True)\
                       ,T.StructField("date_payment", T.TimestampType(), True)
                       ])

df_sp2 = spark \
      .createDataFrame(df_pd2 ,schema=mySchema ) \
      .withColumn('date_violation', F.to_date(F.col('date_violation')) ) \
      .withColumn('date_payment', F.to_date(F.col('date_payment')) ) \
      .withColumn('sum_fine', F.round(F.col('sum_fine')) )

df_sp2.createOrReplaceTempView("fine")

df_sp2.printSchema()
df_sp2.show()

root
 |-- name: string (nullable = true)
 |-- number_plate: string (nullable = true)
 |-- violation: string (nullable = true)
 |-- sum_fine: float (nullable = true)
 |-- date_violation: date (nullable = true)
 |-- date_payment: date (nullable = true)

+-------------+------------+--------------------+--------+--------------+------------+
|         name|number_plate|           violation|sum_fine|date_violation|date_payment|
+-------------+------------+--------------------+--------+--------------+------------+
| Баранов П.Е.|      Р523ВТ|Превышение скорос...|   500.0|    2020-01-12|  2020-01-17|
|Абрамова К.А.|      О111АВ|Проезд на запреща...|  1000.0|    2020-01-14|  2020-02-27|
| Яковлев Г.Р.|      Т330ТТ|Превышение скорос...|   500.0|    2020-01-23|  2020-02-23|
| Яковлев Г.Р.|      М701АА|Превышение скорос...|     NaN|    2020-01-12|        null|
| Колесов С.П.|      К892АХ|Превышение скорос...|     NaN|    2020-02-01|        null|
| Баранов П.Е.|      Р523ВТ|Превышение скорос...|   

In [19]:
t2="""('Превышение скорости(от 20 до 40)', 500),
       ('Превышение скорости(от 40 до 60)', 1000),
       ('Проезд на запрещающий сигнал', 1000)"""

df_pd3 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .replace("'",'') \
                .split(', ')] for x in t2.split('\n')] )#.drop(6,axis=1)

df_pd3.columns = ['violation','sum_fine']

df_pd3['violation'] = df_pd3['violation'].map( lambda x: str(x)[1:] )
df_pd3['sum_fine'] = df_pd3['sum_fine'].map( lambda x: str(x).replace(')','').replace('(','').replace(',','') )
df_pd3['sum_fine'] = pd.to_numeric( df_pd3['sum_fine'], downcast='integer', errors='coerce' )

df_pd3

,violation,sum_fine
0,Превышение скорости(от 20 до 40),500
1,Превышение скорости(от 40 до 60),1000
2,Проезд на запрещающий сигнал,1000


In [20]:
mySchema = StructType([
                       T.StructField("violation", T.StringType(), True)\
                       ,T.StructField("sum_fine", T.IntegerType(), True)
                       ])

df_sp3 = spark \
      .createDataFrame(df_pd3 ,schema=mySchema ) \
      .withColumn('sum_fine', F.round(F.col('sum_fine')) )

df_sp3.createOrReplaceTempView("traffic_violation")

df_sp3.printSchema()
df_sp3.show()

root
 |-- violation: string (nullable = true)
 |-- sum_fine: integer (nullable = true)

+--------------------+--------+
|           violation|sum_fine|
+--------------------+--------+
|Превышение скорос...|     500|
|Превышение скорос...|    1000|
|Проезд на запреща...|    1000|
+--------------------+--------+



## Практика

### Задание_1
Занести в таблицу fine суммы штрафов, которые должен оплатить водитель, в соответствии с данными из таблицы traffic_violation. При этом суммы заносить только в пустые поля столбца  sum_fine.

Таблица traffic_violationсоздана и заполнена.

In [21]:
sql_170_SQL = """UPDATE fine f, traffic_violation tv
SET f.sum_fine = tv.sum_fine
WHERE tv.violation = f.violation and f.sum_fine IS NULL
;
"""
print(sql_170_SQL)



sql_170 = """
select

  f.name
  , f.number_plate
  , f.violation
  , IF(f.sum_fine = 'NaN', tv.sum_fine, f.sum_fine) AS sum_fine
  , f.date_violation
  , f.date_payment

from fine AS f

join traffic_violation AS tv
on tv.violation = f.violation
;

"""

spark.sql(sql_170).toPandas()

UPDATE fine f, traffic_violation tv
SET f.sum_fine = tv.sum_fine
WHERE tv.violation = f.violation and f.sum_fine IS NULL
;



,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500.0,2020-01-12,2020-01-17
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),1000.0,2020-02-14,None
2,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500.0,2020-01-23,2020-02-23
3,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),500.0,2020-01-12,None
4,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),500.0,2020-02-01,None
5,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.0,2020-01-14,2020-02-27
6,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000.0,2020-02-23,None
7,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,1000.0,2020-03-03,None


In [22]:
# Таблица fine изменилась

df_sp4 = spark.sql(sql_170)
df_sp4.createOrReplaceTempView("fine")

df_sp4.printSchema()
df_sp4.show()

root
 |-- name: string (nullable = true)
 |-- number_plate: string (nullable = true)
 |-- violation: string (nullable = true)
 |-- sum_fine: float (nullable = true)
 |-- date_violation: date (nullable = true)
 |-- date_payment: date (nullable = true)

+-------------+------------+--------------------+--------+--------------+------------+
|         name|number_plate|           violation|sum_fine|date_violation|date_payment|
+-------------+------------+--------------------+--------+--------------+------------+
| Баранов П.Е.|      Р523ВТ|Превышение скорос...|   500.0|    2020-01-12|  2020-01-17|
| Баранов П.Е.|      Р523ВТ|Превышение скорос...|  1000.0|    2020-02-14|        null|
| Яковлев Г.Р.|      Т330ТТ|Превышение скорос...|   500.0|    2020-01-23|  2020-02-23|
| Яковлев Г.Р.|      М701АА|Превышение скорос...|   500.0|    2020-01-12|        null|
| Колесов С.П.|      К892АХ|Превышение скорос...|   500.0|    2020-02-01|        null|
|Абрамова К.А.|      О111АВ|Проезд на запреща...|  1

### Задание_2
Вывести фамилию, номер машины и нарушение только для тех водителей, которые на одной машине нарушили одно и то же правило   два и более раз. При этом учитывать все нарушения, независимо от того оплачены они или нет. Информацию отсортировать в алфавитном порядке, сначала по фамилии водителя, потом по номеру машины и, наконец, по нарушению.

In [23]:
sql_171 = """SELECT

    name
    , number_plate
    , violation

FROM fine

GROUP BY 1,2,3

HAVING COUNT(*)>1

ORDER BY 1,2,3

"""

spark.sql(sql_171).toPandas()

,name,number_plate,violation
0,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60)


###Задание_3
В таблице fine увеличить в два раза сумму неоплаченных штрафов для отобранных на предыдущем шаге записей. 

In [24]:
sql_172_SQL = """UPDATE fine f, ( SELECT
                    name
                    , number_plate
                    , violation
                FROM fine
                GROUP BY 1,2,3
                HAVING COUNT(*)>1 ) nn

SET f.sum_fine = f.sum_fine*2

WHERE f.name = nn.name
and f.number_plate = nn.number_plate
and f.violation = nn.violation
and f.date_payment IS NULL
;"""

print(sql_172_SQL)



sql_172 = """
select

  f.name
  , f.number_plate
  , f.violation
  , cast(
         IF( nn.name IS NULL
         , f.sum_fine
         , f.sum_fine*2)
     AS INTEGER) AS sum_fine
  , f.date_violation
  , f.date_payment

from fine AS f

left join ( SELECT
                    name
                    , number_plate
                    , violation
                FROM fine
                GROUP BY 1,2,3
                HAVING COUNT(*)>1 ) nn
on f.name = nn.name
and f.number_plate = nn.number_plate
and f.violation = nn.violation
and f.date_payment is null
;

"""

spark.sql(sql_172).toPandas()

UPDATE fine f, ( SELECT
                    name
                    , number_plate
                    , violation
                FROM fine
                GROUP BY 1,2,3
                HAVING COUNT(*)>1 ) nn

SET f.sum_fine = f.sum_fine*2

WHERE f.name = nn.name
and f.number_plate = nn.number_plate
and f.violation = nn.violation
and f.date_payment IS NULL
;


,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500,2020-01-12,2020-01-17
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),2000,2020-02-14,None
2,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500,2020-01-23,2020-02-23
3,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),500,2020-01-12,None
4,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,1000,2020-03-03,None
5,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000,2020-01-14,2020-02-27
6,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,2000,2020-02-23,None
7,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),500,2020-02-01,None


###Задание_4
Водители оплачивают свои штрафы. В таблице payment занесены даты их оплаты:

<table border="1" cellpadding="0" cellspacing="0">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td><strong>payment_id</strong></td>
			<td><strong>name</strong></td>
			<td><strong>number_plate</strong></td>
			<td><strong>violation</strong></td>
			<td><strong>date_violation</strong></td>
			<td><strong>date_payment</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Яковлев Г.Р.</td>
			<td>М701АА</td>
			<td>Превышение скорости<br>
			(от 20 до 40)</td>
			<td>2020-01-12</td>
			<td>2020-01-22</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Баранов П.Е.</td>
			<td>Р523ВТ</td>
			<td>Превышение скорости<br>
			(от 40 до 60)</td>
			<td>2020-02-14</td>
			<td>2020-03-06</td>
		</tr>
		<tr>
			<td>3</td>
			<td>Яковлев Г.Р.</td>
			<td>Т330ТТ</td>
			<td>Проезд на<br>
			запрещающий сигнал</td>
			<td>2020-03-03</td>
			<td>2020-03-23</td>
		</tr>
	</tbody>
</table>

Необходимо:

в таблицу fine занести дату оплаты соответствующего штрафа из 

*   таблицы payment; 
*   уменьшить начисленный штраф в таблице fine в два раза  (только для тех штрафов, информация о которых занесена в таблицу payment) , если оплата произведена не позднее 20 дней со дня нарушения.

In [25]:
#Загрузим эти данные в Spark SQL

t5 = """('Яковлев Г.Р.', 'М701АА', 'Превышение скорости(от 20 до 40)', '2020-01-12', '2020-01-22'),
('Баранов П.Е.', 'Р523ВТ', 'Превышение скорости(от 40 до 60)', '2020-02-14', '2020-03-06'),
('Яковлев Г.Р.', 'Т330ТТ', 'Проезд на запрещающий сигнал', '2020-03-03', '2020-03-23')"""

df_pd5 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .replace("'",'') \
                .split(', ')] for x in t5.split('\n')] )#.drop(6,axis=1)

df_pd5.columns = ['name', 'number_plate', 'violation', 'date_violation', 'date_payment']

df_pd5['name'] = df_pd5['name'].map( lambda x: str(x).replace(')','').replace('(','') )

df_pd5['date_violation'] = pd.to_datetime(
                               df_pd5['date_violation'].map(
                                            lambda x: str(x).replace(')','').replace('(','').replace(',','').strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='coerce')
df_pd5['date_payment'] = pd.to_datetime(
                               df_pd5['date_payment'].map(
                                            lambda x: str(x).replace(')','').replace('(','').replace(',','').strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='coerce')
df_pd5

,name,number_plate,violation,date_violation,date_payment
0,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),2020-01-12,2020-01-22
1,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),2020-02-14,2020-03-06
2,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,2020-03-03,2020-03-23


In [26]:
mySchema = StructType([
                       T.StructField("name", T.StringType(), True)\
                       ,T.StructField("number_plate", T.StringType(), True)\
                       ,T.StructField("violation", T.StringType(), True)

                       ,T.StructField("date_violation", T.TimestampType(), True)\
                       ,T.StructField("date_payment", T.TimestampType(), True)
                       ])

df_sp5 = spark \
      .createDataFrame(df_pd5 ,schema=mySchema ) \
      .withColumn('date_violation', F.to_date(F.col('date_violation')) ) \
      .withColumn('date_payment', F.to_date(F.col('date_payment')) )

df_sp5.createOrReplaceTempView("payment")

df_sp5.printSchema()
df_sp5.show()

root
 |-- name: string (nullable = true)
 |-- number_plate: string (nullable = true)
 |-- violation: string (nullable = true)
 |-- date_violation: date (nullable = true)
 |-- date_payment: date (nullable = true)

+------------+------------+--------------------+--------------+------------+
|        name|number_plate|           violation|date_violation|date_payment|
+------------+------------+--------------------+--------------+------------+
|Яковлев Г.Р.|      М701АА|Превышение скорос...|    2020-01-12|  2020-01-22|
|Баранов П.Е.|      Р523ВТ|Превышение скорос...|    2020-02-14|  2020-03-06|
|Яковлев Г.Р.|      Т330ТТ|Проезд на запреща...|    2020-03-03|  2020-03-23|
+------------+------------+--------------------+--------------+------------+



In [27]:
sql_173_SQL = """UPDATE fine f, payment p

SET f.date_payment = p.date_payment
    , f.sum_fine = IF(DATEDIFF(p.date_payment,p.date_violation)<=20,f.sum_fine/2, f.sum_fine)

WHERE f.name = p.name
and f.number_plate = p.number_plate
and f.violation = p.violation
and f.date_violation = p.date_violation
and f.date_payment IS NULL
;"""

print(sql_172_SQL)



sql_173 = """
select

  f.name
  , f.number_plate
  , f.violation
  , cast(
         IF( DATEDIFF(p.date_payment,p.date_violation)<=20
            , f.sum_fine/2
            , f.sum_fine )
     AS INTEGER) AS sum_fine
  , f.date_violation
  , IF( p.date_payment IS NULL
        , f.date_payment
        , p.date_payment 
      ) AS date_payment

from fine AS f

left join payment AS p
on f.name = p.name
and f.number_plate = p.number_plate
and f.violation = p.violation
and f.date_violation = p.date_violation
and f.date_payment IS NULL
;

"""

spark.sql(sql_173).toPandas()

UPDATE fine f, ( SELECT
                    name
                    , number_plate
                    , violation
                FROM fine
                GROUP BY 1,2,3
                HAVING COUNT(*)>1 ) nn

SET f.sum_fine = f.sum_fine*2

WHERE f.name = nn.name
and f.number_plate = nn.number_plate
and f.violation = nn.violation
and f.date_payment IS NULL
;


,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),1000,2020-02-14,2020-03-06
1,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000,2020-01-14,2020-02-27
2,Яковлев Г.Р.,М701АА,Превышение скорости(от 20 до 40),250,2020-01-12,2020-01-22
3,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000,2020-02-23,None
4,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),500,2020-01-12,2020-01-17
5,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),500,2020-02-01,None
6,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,500,2020-03-03,2020-03-23
7,Яковлев Г.Р.,Т330ТТ,Превышение скорости(от 20 до 40),500,2020-01-23,2020-02-23


In [28]:
# Таблица fine изменилась

df_sp6 = spark.sql(sql_173)
df_sp6.createOrReplaceTempView("fine")

df_sp6.printSchema()
df_sp6.show()

root
 |-- name: string (nullable = true)
 |-- number_plate: string (nullable = true)
 |-- violation: string (nullable = true)
 |-- sum_fine: integer (nullable = true)
 |-- date_violation: date (nullable = true)
 |-- date_payment: date (nullable = true)

+-------------+------------+--------------------+--------+--------------+------------+
|         name|number_plate|           violation|sum_fine|date_violation|date_payment|
+-------------+------------+--------------------+--------+--------------+------------+
| Баранов П.Е.|      Р523ВТ|Превышение скорос...|    1000|    2020-02-14|  2020-03-06|
|Абрамова К.А.|      О111АВ|Проезд на запреща...|    1000|    2020-01-14|  2020-02-27|
| Яковлев Г.Р.|      М701АА|Превышение скорос...|     250|    2020-01-12|  2020-01-22|
|Абрамова К.А.|      О111АВ|Проезд на запреща...|    1000|    2020-02-23|        null|
| Баранов П.Е.|      Р523ВТ|Превышение скорос...|     500|    2020-01-12|  2020-01-17|
| Колесов С.П.|      К892АХ|Превышение скорос...| 

###Задание_5
Создать новую таблицу back_payment, куда внести информацию о неоплаченных штрафах (Фамилию и инициалы водителя, номер машины, нарушение, сумму штрафа  и  дату нарушения) из таблицы fine.

In [29]:
sql_174_SQL = """CREATE TABLE back_payment
(
  SELECT

      f.name
      , f.number_plate
      , f.violation
      , f.sum_fine
      , f.date_violation

    FROM fine AS f

    WHERE f.date_payment IS NULL      
);

SELECT * FROM back_payment;
"""
print(sql_174_SQL)



sql_174 = """SELECT

      f.name
      , f.number_plate
      , f.violation
      , f.sum_fine
      , f.date_violation

    FROM fine AS f

    WHERE f.date_payment IS NULL """

df_sp7 = spark.sql(sql_174)
df_sp7.createOrReplaceTempView("back_payment")

spark.sql('select * from back_payment').toPandas()

CREATE TABLE back_payment
(
  SELECT

      f.name
      , f.number_plate
      , f.violation
      , f.sum_fine
      , f.date_violation

    FROM fine AS f

    WHERE f.date_payment IS NULL      
);

SELECT * FROM back_payment;



,name,number_plate,violation,sum_fine,date_violation
0,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000,2020-02-23
1,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),500,2020-02-01


###Задание_6
Удалить из таблицы fine информацию о нарушениях, совершенных раньше 1 февраля 2020 года. 

In [30]:
sql_175_SQL = """DELETE FROM fine 
WHERE date_violation < DATE'2020-02-01'
;
    
SELECT * FROM fine;"""
print(sql_175_SQL)



sql_175 = """
SELECT

      f.name
      , f.number_plate
      , f.violation
      , f.sum_fine
      , f.date_violation
      , f.date_payment

    FROM fine AS f
    
    WHERE f.date_violation >= DATE'2020-02-01'
"""

df_sp7 = spark.sql(sql_175)
df_sp7.createOrReplaceTempView("fine")

spark.sql('select * from fine').toPandas()

DELETE FROM fine 
WHERE date_violation < DATE'2020-02-01'
;
    
SELECT * FROM fine;


,name,number_plate,violation,sum_fine,date_violation,date_payment
0,Баранов П.Е.,Р523ВТ,Превышение скорости(от 40 до 60),1000,2020-02-14,2020-03-06
1,Абрамова К.А.,О111АВ,Проезд на запрещающий сигнал,1000,2020-02-23,None
2,Колесов С.П.,К892АХ,Превышение скорости(от 20 до 40),500,2020-02-01,None
3,Яковлев Г.Р.,Т330ТТ,Проезд на запрещающий сигнал,500,2020-03-03,2020-03-23


### Примечание

<span><h2>Поиск по ключевым словам</h2>

<p>На данном шаге можно найти шаги курса, в которых встречаются ключевые слова SQL, которые рассматриваются в курсе.</p>

<p>Для этого скопируйте&nbsp;один из запросов&nbsp;в окно решений, укажите нужные ключевые слова и запустите запрос. В окне решений будут выведены ссылки на соответствующие шаги.</p>

<p>Это <strong>НЕ ЗАДАНИЕ</strong>, а просто запросы, с помощью которых можно найти шаги, в которых встречаются те или иные ключевые слова. Выполнять не обязательно (это задание оценивается в 0 баллов). Это ПРОСТО ПОМОЩЬ для навигации по курсу.</p>

<p><strong>Запрос 1.&nbsp;</strong>Поиск шагов,&nbsp;в которых встречается заданное ключевое слово, в примере <strong>MAX</strong>:</p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> 
   <span class="hljs-keyword">CONCAT</span>(module_id,<span class="hljs-string">'.'</span>,lesson_position,<span class="hljs-string">"."</span>,step_position,<span class="hljs-string">" "</span>, <span class="hljs-keyword">CONCAT</span>(<span class="hljs-keyword">LEFT</span>(step_name, <span class="hljs-number">50</span>), <span class="hljs-string">'...'</span>)) <span class="hljs-keyword">AS</span> Шаг,
   note <span class="hljs-keyword">AS</span> Примечание
<span class="hljs-keyword">FROM</span> step
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> lesson <span class="hljs-keyword">USING</span>(lesson_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> <span class="hljs-keyword">module</span> <span class="hljs-keyword">USING</span>(module_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> step_keyword <span class="hljs-keyword">USING</span>(step_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> keyword <span class="hljs-keyword">USING</span>(keyword_id)
<span class="hljs-keyword">WHERE</span> keyword_name = <span class="hljs-string">'MAX'</span>
<span class="hljs-keyword">ORDER</span> <span class="hljs-keyword">BY</span> <span class="hljs-number">1</span>;</code></pre>

<p><strong>Запрос 2.&nbsp;</strong>Поиск шагов,&nbsp;в которых встречаются два заданных ключевых слова одновременно, в примере <strong>MAX</strong> и <strong>AVG</strong>:</p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> 
   <span class="hljs-keyword">CONCAT</span>(module_id,<span class="hljs-string">'.'</span>,lesson_position,<span class="hljs-string">"."</span>,step_position,<span class="hljs-string">" "</span>, <span class="hljs-keyword">CONCAT</span>(<span class="hljs-keyword">LEFT</span>(step_name, <span class="hljs-number">30</span>), <span class="hljs-string">'...'</span>)) <span class="hljs-keyword">AS</span> Шаг, 
   <span class="hljs-keyword">link</span> <span class="hljs-keyword">AS</span> Ссылка_на_шаг
<span class="hljs-keyword">FROM</span> step
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> lesson <span class="hljs-keyword">USING</span>(lesson_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> <span class="hljs-keyword">module</span> <span class="hljs-keyword">USING</span>(module_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> step_keyword <span class="hljs-keyword">USING</span>(step_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> keyword <span class="hljs-keyword">USING</span>(keyword_id)
<span class="hljs-keyword">WHERE</span> keyword_name <span class="hljs-keyword">IN</span> (<span class="hljs-string">'MAX'</span>, <span class="hljs-string">'AVG'</span>)
<span class="hljs-keyword">GROUP</span> <span class="hljs-keyword">BY</span> ШАГ, Ссылка_на_шаг
<span class="hljs-keyword">HAVING</span> <span class="hljs-keyword">count</span>(*) = <span class="hljs-number">2</span>
<span class="hljs-keyword">ORDER</span> <span class="hljs-keyword">BY</span> <span class="hljs-number">1</span>;</code></pre>

<p><strong>&nbsp;Запрос 3.&nbsp;</strong>Поиск шагов,&nbsp;в которых встречаются три заданных ключевых слова одновременно, в примере <strong>MAX</strong>, <strong>MIN</strong> и <strong>AVG</strong>:</p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">SELECT</span> 
   <span class="hljs-keyword">CONCAT</span>(module_id,<span class="hljs-string">'.'</span>,lesson_position,<span class="hljs-string">"."</span>,step_position,<span class="hljs-string">" "</span>,step_name) <span class="hljs-keyword">AS</span> Шаг
<span class="hljs-keyword">FROM</span> step
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> lesson <span class="hljs-keyword">USING</span>(lesson_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> <span class="hljs-keyword">module</span> <span class="hljs-keyword">USING</span>(module_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> step_keyword <span class="hljs-keyword">USING</span>(step_id)
        <span class="hljs-keyword">INNER</span> <span class="hljs-keyword">JOIN</span> keyword <span class="hljs-keyword">USING</span>(keyword_id)
<span class="hljs-keyword">WHERE</span> keyword_name <span class="hljs-keyword">IN</span> (<span class="hljs-string">'MAX'</span>, <span class="hljs-string">'AVG'</span>, <span class="hljs-string">'MIN'</span>)
<span class="hljs-keyword">GROUP</span> <span class="hljs-keyword">BY</span> ШАГ
<span class="hljs-keyword">HAVING</span> <span class="hljs-keyword">COUNT</span>(*) = <span class="hljs-number">3</span>
<span class="hljs-keyword">ORDER</span> <span class="hljs-keyword">BY</span> <span class="hljs-number">1</span>;</code></pre></span>

## 2.1 Связи между таблицами

## Практика

In [31]:
#Загрузим эти данные в Spark SQL

sql_210 = """INSERT INTO author(name_author)
VALUES
    ('Булгаков М.А.'),
    ('Достоевский Ф.М.'),
    ('Есенин С.А.'),
    ('Пастернак Б.Л.')
;"""
print(sql_210)



t6 = """('Булгаков М.А.'),
       ('Достоевский Ф.М.'),
       ('Есенин С.А.'),
       ('Пастернак Б.Л.')"""

df_pd8 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("'")] for x in t6.split('\n')] ).drop([0,2],axis=1)
 
df_pd8 = df_pd8.reset_index()
df_pd8.columns = ['author_id','name_author']
df_pd8['author_id'] = df_pd8['author_id']+1

df_pd8

INSERT INTO author(name_author)
VALUES
    ('Булгаков М.А.'),
    ('Достоевский Ф.М.'),
    ('Есенин С.А.'),
    ('Пастернак Б.Л.')
;


,author_id,name_author
0,1,Булгаков М.А.
1,2,Достоевский Ф.М.
2,3,Есенин С.А.
3,4,Пастернак Б.Л.


In [32]:
mySchema = StructType([
                        T.StructField("author_id", T.IntegerType(), True)
                       , T.StructField("name_author", T.StringType(), True)\
                       ])

pDF_to_sDF( df_input=df_pd8
           , table_name="author"
           , mySchema=mySchema )

root
 |-- author_id: integer (nullable = true)
 |-- name_author: string (nullable = true)

+---------+----------------+
|author_id|     name_author|
+---------+----------------+
|        1|   Булгаков М.А.|
|        2|Достоевский Ф.М.|
|        3|     Есенин С.А.|
|        4|  Пастернак Б.Л.|
+---------+----------------+



DataFrame[author_id: int, name_author: string]

### Задание_1
<p>Перепишите&nbsp;запрос на создание таблицы&nbsp;<code><strong>book</strong></code>&nbsp;, чтобы ее структура соответствовала структуре, показанной на логической схеме (таблица <code><strong>genre</strong></code> уже создана, порядок следования столбцов - как на логической схеме в таблице <code><strong>book</strong></code>, <code><strong>genre_id</strong></code>&nbsp; - внешний ключ)&nbsp;. Для <strong><code>genre_id</code></strong>&nbsp;ограничение о недопустимости пустых значений <strong>не задавать</strong>. В качестве главной таблицы для описания поля&nbsp; <strong><code>genre_id</code></strong>использовать таблицу <a href="https://stepik.org/lesson/297508/step/5?unit=279268" rel="noopener noreferrer nofollow"><code><strong>genre</strong></code></a> следующей структуры:</p>

<table border="1" cellpadding="1" cellspacing="1">
	<tbody>
		<tr style="background-color: #e6f8e0; background: #e6f8e0; text-align: center;">
			<td style="text-align: center;"><strong>Поле</strong></td>
			<td style="text-align: center;"><strong>Тип, описание</strong></td>
		</tr>
		<tr>
			<td><code>genre_id</code></td>
			<td><code>INT PRIMARY KEY AUTO_INCREMENT</code></td>
		</tr>
		<tr>
			<td><code>name_genre</code></td>
			<td><code>VARCHAR(30)</code></td>
		</tr>
	</tbody>
</table>

<p><strong>Логическая схема</strong> (нужно создать только таблицу <strong><code>book</code></strong>):</p>

<p><img alt="" src="https://ucarecdn.com/95045d96-412d-4e10-88f2-7ac6b13fada6/"></p>

In [33]:
sql_211 = """CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT, 
    title VARCHAR(50), 
    author_id INT NOT NULL, 
    genre_id INT, 
    
    price DECIMAL(8,2), 
    amount INT, 
    FOREIGN KEY (genre_id)  REFERENCES genre (genre_id) ,
    FOREIGN KEY (author_id)  REFERENCES author (author_id) 
);"""
print(sql_211)

CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT, 
    title VARCHAR(50), 
    author_id INT NOT NULL, 
    genre_id INT, 
    
    price DECIMAL(8,2), 
    amount INT, 
    FOREIGN KEY (genre_id)  REFERENCES genre (genre_id) ,
    FOREIGN KEY (author_id)  REFERENCES author (author_id) 
);


### Задание_2
<p>Создать таблицу <code><strong>book</strong></code> той же структуры, что и на предыдущем шаге.&nbsp;Будем считать, что при удалении автора из таблицы <code><strong>author</strong></code>, должны удаляться все записи о книгах из таблицы <code><strong>book</strong></code>, написанные этим автором. А при удалении жанра из таблицы <code><strong>genre</strong></code> для соответствующей записи <code><strong>book</strong></code> установить значение <code><strong>Null </strong></code>в столбце <code><strong>genre_id</strong></code>.&nbsp;</p>

<span><h2>Действия при&nbsp;удалении записи главной таблицы</h2>

<p>С помощью выражения<strong>&nbsp;</strong><code>ON DELETE</code>&nbsp;можно установить действия, которые выполняются для записей подчиненной таблицы при удалении связанной строки из главной таблицы. При удалении можно установить следующие опции:</p>

<ul>
	<li><code>CASCADE</code>: автоматически удаляет строки из зависимой таблицы при удалении&nbsp; связанных строк в главной таблице.</li>
	<li><code>SET NULL</code>: при удалении&nbsp; связанной строки из главной таблицы устанавливает для столбца внешнего ключа значение&nbsp;<strong><code>NULL</code></strong>. (В этом случае столбец внешнего ключа должен поддерживать установку<code><strong> NULL</strong></code>).</li>
	<li><code>SET&nbsp;DEFAULT</code>&nbsp;похоже на <code>SET&nbsp;NULL</code> за тем исключением, что значение&nbsp; внешнего ключа устанавливается не в <code>NULL,</code>&nbsp;а в значение по умолчанию для данного столбца.</li>
	<li><code>RESTRICT</code>: отклоняет удаление строк в главной таблице при наличии связанных строк в зависимой таблице.</li>
</ul>

<p><strong>Важно! </strong>Если для столбца установлена опция <code>SET NULL</code>, то при его описании нельзя задать ограничение на пустое значение.</p>

In [34]:
sql_122 = """CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT, 
    title VARCHAR(50), 
    author_id INT NOT NULL, 
    genre_id INT, 
    
    price DECIMAL(8,2), 
    amount INT, 
    FOREIGN KEY (genre_id)  REFERENCES genre (genre_id) ON DELETE SET NULL,
    FOREIGN KEY (author_id)  REFERENCES author (author_id)  ON DELETE CASCADE
);"""
print(sql_122)

CREATE TABLE book (
    book_id INT PRIMARY KEY AUTO_INCREMENT, 
    title VARCHAR(50), 
    author_id INT NOT NULL, 
    genre_id INT, 
    
    price DECIMAL(8,2), 
    amount INT, 
    FOREIGN KEY (genre_id)  REFERENCES genre (genre_id) ON DELETE SET NULL,
    FOREIGN KEY (author_id)  REFERENCES author (author_id)  ON DELETE CASCADE
);


### Задание_3

<div id="ember15823" class="html-content rich-text-viewer ember-view" data-processed=""><!----><span><h2>Заполнение таблицы с внешними ключами</h2>

<p>На предыдущих шагах были созданы и заполнены таблицы<code><strong> author</strong></code>:</p>

<table border="1" cellpadding="1" cellspacing="1">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td style="text-align: center;"><strong>author_id</strong></td>
			<td style="text-align: center;"><strong>name_author</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Булгаков М.А.</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Достоевский Ф.М.</td>
		</tr>
		<tr>
			<td>3</td>
			<td>Есенин С.А.</td>
		</tr>
		<tr>
			<td>4</td>
			<td>Пастернак Б.Л.</td>
		</tr>
	</tbody>
</table>

<p>и&nbsp;&nbsp;<code><strong>genre</strong></code>:</p>

<table border="1" cellpadding="1" cellspacing="1">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td style="text-align: center;"><strong>genre_id</strong></td>
			<td style="text-align: center;"><strong>name_genre</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Роман</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Поэзия</td>
		</tr>
	</tbody>
</table>


<h2>Задание</h2>

<p>Для каждой строки таблицы <code><strong>book</strong></code> занесите значения в поля <code><strong>author_id </strong></code>и <code><strong>genre_id</strong></code>. Считать, что книга Есенина относится к жанру «Поэзия», остальные книги – к жанру «Роман».</p>

<p>Через запятую перечислены&nbsp;значения полей&nbsp;<code><strong>book_id</strong></code>,&nbsp; <code><strong>title</strong></code>, <code><strong>author_id</strong></code>, <code><strong>genre_id</strong></code>,&nbsp;&nbsp;<code><strong>price</strong></code>, <code><strong>amount</strong></code> каждой записи таблицы <code><strong>book</strong></code>. Заполните пропуски.</p>

<p>Авторы и их произведения:</p>

<table border="1" cellpadding="0" cellspacing="0">
	<tbody>
		<tr style="background-color: #dcdcdc; background: #dcdcdc; text-align: center;">
			<td>
			<strong>Название книги</strong>
			</td>
			<td>
			<strong>Автор</strong>
			</td>
			<td>
			<strong>Цена</strong>
			</td>
			<td>
			<strong>Количество</strong>
			</td>
		</tr>
		<tr>
			<td>Мастер и Маргарита</td>
			<td>Булгаков М.А.</td>
			<td>670.99</td>
			<td>3</td>
		</tr>
		<tr>
			<td>Белая гвардия</td>
			<td>Булгаков М.А.</td>
			<td>540.50</td>
			<td>5</td>
		</tr>
		<tr>
			<td>Идиот</td>
			<td>Достоевский Ф.М.</td>
			<td>460.00</td>
			<td>10</td>
		</tr>
		<tr>
			<td>Братья Карамазовы</td>
			<td>Достоевский Ф.М.</td>
			<td>799.01</td>
			<td>3</td>
		</tr>
		<tr>
			<td>Игрок</td>
			<td>Достоевский Ф.М.</td>
			<td>480.50</td>
			<td>10</td>
		</tr>
		<tr>
			<td>Стихотворения и поэмы</td>
			<td>Есенин С.А.</td>
			<td>650.00</td>
			<td>15</td>
		</tr>
	</tbody>
</table></span></div>

### Задание_4
<p>Добавьте три последние записи (с ключевыми значениями 6, 7, 8) в таблицу<code><strong> book</strong></code>, первые 5 записей уже добавлены:</p>

<table border="1" cellpadding="0" cellspacing="0">
	<tbody>
		<tr style="background-color: #a9a9a9; background: #a9a9a9; text-align: center;">
			<td><strong>book_id</strong></td>
			<td><strong>title</strong></td>
			<td><strong>author_id</strong></td>
			<td><strong>genre_id</strong></td>
			<td><strong>price</strong></td>
			<td><strong>amount</strong></td>
		</tr>
		<tr>
			<td>1</td>
			<td>Мастер и Маргарита</td>
			<td>1</td>
			<td>1</td>
			<td>670.99</td>
			<td>3</td>
		</tr>
		<tr>
			<td>2</td>
			<td>Белая гвардия</td>
			<td>1</td>
			<td>1</td>
			<td>540.50</td>
			<td>5</td>
		</tr>
		<tr>
			<td>3</td>
			<td>Идиот</td>
			<td>2</td>
			<td>1</td>
			<td>460.00</td>
			<td>10</td>
		</tr>
		<tr>
			<td>4</td>
			<td>Братья Карамазовы</td>
			<td>2</td>
			<td>1</td>
			<td>799.01</td>
			<td>3</td>
		</tr>
		<tr>
			<td>5</td>
			<td>Игрок</td>
			<td>2</td>
			<td>1</td>
			<td>480.50</td>
			<td>10</td>
		</tr>
		<tr>
			<td>6</td>
			<td>Стихотворения и поэмы</td>
			<td>3</td>
			<td>2</td>
			<td>650.00</td>
			<td>15</td>
		</tr>
		<tr>
			<td>7</td>
			<td>Черный человек</td>
			<td>3</td>
			<td>2</td>
			<td>570.20</td>
			<td>6</td>
		</tr>
		<tr>
			<td>8</td>
			<td>Лирика</td>
			<td>4</td>
			<td>2</td>
			<td>518.99</td>
			<td>2</td>
		</tr>
	</tbody>
</table>

<p><strong>Логическая схема базы данных:</strong></p>

<p><img alt="" src="https://ucarecdn.com/26d5d90a-3e95-46bc-807d-10fb53d10f25/"></p>

<details><summary><strong>Результат</strong></summary>

<pre><code class="language-sql hljs">Affected rows: 1
Affected rows: 1
Affected rows: 1
Query result:
+<span class="hljs-comment">---------+-----------------------+-----------+----------+--------+--------+</span>
| book_id | title                 | author_id | genre_id | price  | amount |
+<span class="hljs-comment">---------+-----------------------+-----------+----------+--------+--------+</span>
| 1       | Мастер и Маргарита    | 1         | 1        | 670.99 | 3      |
| 2       | Белая гвардия         | 1         | 1        | 540.50 | 5      |
| 3       | Идиот                 | 2         | 1        | 460.00 | 10     |
| 4       | Братья Карамазовы     | 2         | 1        | 799.01 | 3      |
| 5       | Игрок                 | 2         | 1        | 480.50 | 10     |
| 6       | Стихотворения и поэмы | 3         | 2        | 650.00 | 15     |
| 7       | Черный человек        | 3         | 2        | 570.20 | 6      |
| 8       | Лирика                | 4         | 2        | 518.99 | 2      |
+<span class="hljs-comment">---------+-----------------------+-----------+----------+--------+--------+</span></code></pre>
</details></span>

In [35]:
sql_123 = """INSERT INTO book(title,author_id,genre_id,price,amount)
VALUES     ('Стихотворения и поэмы',3,2,650.00,15),
        ('Черный человек',3,2,570.20,6),
        ('Лирика',4,2,518.99,2)
;"""
print(sql_123)

INSERT INTO book(title,author_id,genre_id,price,amount)
VALUES     ('Стихотворения и поэмы',3,2,650.00,15),
        ('Черный человек',3,2,570.20,6),
        ('Лирика',4,2,518.99,2)
;


### Задание_5
Вывести название, жанр и цену тех книг, количество которых больше 8, в отсортированном по убыванию цены виде.

In [36]:
t7 = """1	Мастер и Маргарита	1	1	670.99	3
2	Белая гвардия	1	1	540.50	5
3	Идиот	2	1	460.00	10
4	Братья Карамазовы	2	1	799.01	3
5	Игрок	2	1	480.50	10
6	Стихотворения и поэмы	3	2	650.00	15
7	Черный человек	3	2	570.20	6
8	Лирика	4	2	518.99	2"""

df_pd9 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t7.split('\n')] ).drop([0],axis=1)
 
df_pd9.columns = 'title	author_id	genre_id	price	amount'.split('\t')
df_pd9['author_id'] = df_pd9['author_id'].astype('int32')
df_pd9['genre_id'] = df_pd9['genre_id'].astype('int32')
df_pd9['price'] = df_pd9['price'].astype('float64')
df_pd9['amount'] = df_pd9['amount'].astype('int32')

df_pd9

,title,author_id,genre_id,price,amount
0,Мастер и Маргарита,1,1,670.99,3
1,Белая гвардия,1,1,540.50,5
2,Идиот,2,1,460.00,10
3,Братья Карамазовы,2,1,799.01,3
4,Игрок,2,1,480.50,10
5,Стихотворения и поэмы,3,2,650.00,15
6,Черный человек,3,2,570.20,6
7,Лирика,4,2,518.99,2


In [37]:
mySchema = StructType([
                        T.StructField("title", T.StringType(), True)
                       
                       , T.StructField("author_id", T.IntegerType(), True)\
                       , T.StructField("genre_id", T.IntegerType(), True)\

                       , T.StructField("price", T.FloatType(), True)\
                       
                       , T.StructField("amount", T.IntegerType(), True)\
                       ])
pDF_to_sDF( df_input=df_pd9
           , table_name="book"
           , mySchema=mySchema )

root
 |-- title: string (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- genre_id: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- amount: integer (nullable = true)

+--------------------+---------+--------+------+------+
|               title|author_id|genre_id| price|amount|
+--------------------+---------+--------+------+------+
|  Мастер и Маргарита|        1|       1|670.99|     3|
|       Белая гвардия|        1|       1| 540.5|     5|
|               Идиот|        2|       1| 460.0|    10|
|   Братья Карамазовы|        2|       1|799.01|     3|
|               Игрок|        2|       1| 480.5|    10|
|Стихотворения и п...|        3|       2| 650.0|    15|
|      Черный человек|        3|       2| 570.2|     6|
|              Лирика|        4|       2|518.99|     2|
+--------------------+---------+--------+------+------+



DataFrame[title: string, author_id: int, genre_id: int, price: float, amount: int]

In [38]:
t7 = """1	Роман
2	Поэзия"""

df_pd10 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t7.split('\n')] )
 
df_pd10.columns = 'genre_id	name_genre'.split('\t')
for col in [x for x in df_pd10.columns if '_id' in x]:
  df_pd10[col] = df_pd10[col].astype('int64')

df_pd10

,genre_id,name_genre
0,1,Роман
1,2,Поэзия


In [39]:
mySchema = StructType([
                       T.StructField("genre_id", T.IntegerType(), True)\
                       , T.StructField("name_genre", T.StringType(), True)
                       ])
pDF_to_sDF( df_input=df_pd10
           , table_name="genre"
           , mySchema=mySchema )

root
 |-- genre_id: integer (nullable = true)
 |-- name_genre: string (nullable = true)

+--------+----------+
|genre_id|name_genre|
+--------+----------+
|       1|     Роман|
|       2|    Поэзия|
+--------+----------+



DataFrame[genre_id: int, name_genre: string]

In [40]:
sql_124 = """SELECT 
    book.title
    , genre.name_genre
    , book.price
FROM author 
INNER JOIN book
ON author.author_id = book.author_id
INNER JOIN genre
ON genre.genre_id = book.genre_id
WHERE book.amount > 8
ORDER BY book.price DESC
;"""

spark.sql(sql_124).toPandas()

,title,name_genre,price
0,Стихотворения и поэмы,Поэзия,650.0
1,Игрок,Роман,480.5
2,Идиот,Роман,460.0


### Задание_6

Вывести все жанры, которые не представлены в книгах на складе.

In [41]:
# Добавить жанр "Приключения"

mySchema = StructType([
                       T.StructField("genre_id", T.IntegerType(), True)\
                       , T.StructField("name_genre", T.StringType(), True)
                       ])

df_sp10 = pDF_to_sDF( df_input=df_pd10
           , table_name="genre"
           , mySchema=mySchema )

newRow = spark.createDataFrame([(3,'Приключения')], df_sp10.columns)
appended = df_sp10.union(newRow)
appended.show()

pDF_to_sDF( df_input=appended.toPandas()
           , table_name="genre"
           , mySchema=mySchema )

root
 |-- genre_id: integer (nullable = true)
 |-- name_genre: string (nullable = true)

+--------+----------+
|genre_id|name_genre|
+--------+----------+
|       1|     Роман|
|       2|    Поэзия|
+--------+----------+

+--------+-----------+
|genre_id| name_genre|
+--------+-----------+
|       1|      Роман|
|       2|     Поэзия|
|       3|Приключения|
+--------+-----------+

root
 |-- genre_id: integer (nullable = true)
 |-- name_genre: string (nullable = true)

+--------+-----------+
|genre_id| name_genre|
+--------+-----------+
|       1|      Роман|
|       2|     Поэзия|
|       3|Приключения|
+--------+-----------+



DataFrame[genre_id: int, name_genre: string]

In [42]:
sql_125 = """SELECT 
    genre.name_genre
    
FROM genre

LEFT JOIN book
ON genre.genre_id = book.genre_id

WHERE book.title IS NULL
;"""

spark.sql(sql_125).toPandas()

,name_genre
0,Приключения


### Задание_7
Необходимо в каждом городе провести выставку книг каждого автора в течение 2020 года. Дату проведения выставки выбрать случайным образом. Создать запрос, который выведет город, автора и дату проведения выставки. Последний столбец назвать Дата. Информацию вывести, отсортировав сначала в алфавитном порядке по названиям городов, а потом по убыванию дат проведения выставок.

<details open=""><summary><strong>Пояснение</strong></summary>

<p>1. Для генерации случайной даты можно к&nbsp;первому числу года ('2020-01-01') прибавить целое случайное число в интервале от 0 до 365.</p>

<p>Генерации случайных чисел в интервале от 0 до 1 (не включительно)&nbsp;осуществляется с помощью функции <code>RAND()</code>. Если эту функцию умножить на 365, то она будет генерировать вещественные числа от 0 до 365 (не включительно). Осталось только отбросить дробную часть. Это можно сделать с помощью функции <code>FLOOR()</code>, которая возвращает наибольшее целое число, меньшее или равное указанному числовому значению. Таким образом, случайное число от 0 до 365 можно получить с помощью выражения:</p>

<pre><code class="hljs excel"><span class="hljs-built_in">FLOOR</span>(<span class="hljs-built_in">RAND</span>() * <span class="hljs-number">365</span>)</code></pre>

<p><strong>Важно! </strong>Даты должны быть за 2020 год, первое число года - 1 января 2020 года.</p>

<p>2. Для сложения&nbsp; даты с числом используется функция:</p>

<pre><code class="language-sql hljs">DATE_ADD(дата, INTERVAL число единица_измерения),

где
  единица_измерения (использовать прописные буквы) – это день (DAY), месяц(MONTH), неделя(WEEK) и пр., 
  число – целое число,
  дата – значение даты или даты и времени.</code></pre>

<p>Функция к <strong>дате</strong>&nbsp; прибавляет указанное <strong>число</strong>, выраженное в днях, месяцах и пр. , в зависимости от заданного интервала, и возвращает новую дату.</p>

<p>Например:</p>

<pre><code class="language-sql hljs">DATE_ADD('2020-02-02', INTERVAL 45 DAY) возвращает 18 марта 2020 года
DATE_ADD('2020-02-02', INTERVAL 6 MONTH) возвращает 2 августа 2020 года</code></pre>
</details>

In [43]:
# Создадим таблицу city

t8 = """1	Москва
2	Санкт-Петербург
3	Владивосток"""

df_pd11 = pd.DataFrame( [ 
            [y.strip() for y in x \
                .split("\t")] for x in t8.split('\n')] )
 
df_pd11.columns = 'city_id	name_city'.split('\t')

for col in [x for x in df_pd11.columns if '_id' in x]:
  df_pd11[col] = df_pd11[col].astype('int64')

df_pd11

,city_id,name_city
0,1,Москва
1,2,Санкт-Петербург
2,3,Владивосток


In [44]:
mySchema = StructType([
                       T.StructField("city_id", T.IntegerType(), True)\
                       , T.StructField("name_city", T.StringType(), True)
                       ])
pDF_to_sDF( df_input=df_pd11
           , table_name="city"
           , mySchema=mySchema )

root
 |-- city_id: integer (nullable = true)
 |-- name_city: string (nullable = true)

+-------+---------------+
|city_id|      name_city|
+-------+---------------+
|      1|         Москва|
|      2|Санкт-Петербург|
|      3|    Владивосток|
+-------+---------------+



DataFrame[city_id: int, name_city: string]

In [61]:
sql_126_SQL = """SELECT

    city.name_city
    , author.name_author
    , DATE_ADD('2020-01-01', INTERVAL FLOOR(RAND() * 365) DAY) AS date_dt

FROM city

CROSS JOIN author
ON 1=1

ORDER BY city.name_city ASC
    , date_dt DESC
;"""
print(sql_126_SQL)



sql_126 = """SELECT

    city.name_city
    , author.name_author
    , DATE_ADD( DATE ('2020-01-01'), CAST( FLOOR(RAND()*365) AS INTEGER) ) AS date_dt

FROM city

CROSS JOIN author
ON 1=1

ORDER BY city.name_city ASC
    , date_dt DESC
;"""
spark.sql(sql_126).toPandas()

SELECT

    city.name_city
    , author.name_author
    , DATE_ADD('2020-01-01', INTERVAL FLOOR(RAND() * 365) DAY) AS date_dt

FROM city

CROSS JOIN author
ON 1=1

ORDER BY city.name_city ASC
    , date_dt DESC
;


,name_city,name_author,date_dt
0,Владивосток,Достоевский Ф.М.,2020-09-21
1,Владивосток,Пастернак Б.Л.,2020-06-28
2,Владивосток,Есенин С.А.,2020-06-21
3,Владивосток,Булгаков М.А.,2020-05-12
4,Москва,Достоевский Ф.М.,2020-06-26
5,Москва,Есенин С.А.,2020-04-25
6,Москва,Булгаков М.А.,2020-03-16
7,Москва,Пастернак Б.Л.,2020-02-14
8,Санкт-Петербург,Пастернак Б.Л.,2020-08-30
9,Санкт-Петербург,Достоевский Ф.М.,2020-04-23


### Задание_8
Вывести информацию о книгах (жанр, книга, автор), относящихся к жанру, включающему слово «роман» в отсортированном по названиям книг виде.

In [63]:
sql_127 = """SELECT

    genre.name_genre
    , book.title
    , author.name_author

FROM author

INNER JOIN book
ON author.author_id = book.author_id

INNER JOIN genre
ON genre.genre_id = book.genre_id

WHERE lower(genre.name_genre) LIKE '%роман%'

ORDER BY book.title ASC
;"""
spark.sql(sql_127).toPandas()

,name_genre,title,name_author
0,Роман,Белая гвардия,Булгаков М.А.
1,Роман,Братья Карамазовы,Достоевский Ф.М.
2,Роман,Игрок,Достоевский Ф.М.
3,Роман,Идиот,Достоевский Ф.М.
4,Роман,Мастер и Маргарита,Булгаков М.А.


### Задание_9
<p>Посчитать количество экземпляров&nbsp; книг каждого автора из таблицы <code><strong>author</strong></code>.&nbsp; Вывести тех авторов,&nbsp;&nbsp;количество книг которых меньше 10, в отсортированном по возрастанию количества виде. Последний столбец назвать&nbsp;<code><strong>Количество</strong></code>.</p>

In [79]:
mySchema = StructType([
                        T.StructField("author_id", T.IntegerType(), True)
                       , T.StructField("name_author", T.StringType(), True)\
                       ])

pDF_to_sDF( df_input = df_pd8.append(
                          pd.DataFrame( [(5,'Лермонтов М.Ю.')]
                                        , columns=df_pd8.columns)
                          )
           , table_name="author"
           , mySchema=mySchema )

root
 |-- author_id: integer (nullable = true)
 |-- name_author: string (nullable = true)

+---------+----------------+
|author_id|     name_author|
+---------+----------------+
|        1|   Булгаков М.А.|
|        2|Достоевский Ф.М.|
|        3|     Есенин С.А.|
|        4|  Пастернак Б.Л.|
|        5|  Лермонтов М.Ю.|
+---------+----------------+



DataFrame[author_id: int, name_author: string]

In [80]:
sql_128 = """SELECT 
    name_author
    , sum(amount) AS Kolichestvo

FROM author 

LEFT JOIN book
    on author.author_id = book.author_id

GROUP BY name_author

HAVING ( Coalesce(sum(amount),0) < 10 ) 

ORDER BY Kolichestvo ASC
;"""

spark.sql(sql_128).toPandas()

,name_author,Kolichestvo
0,Лермонтов М.Ю.,NaN
1,Пастернак Б.Л.,2.0
2,Булгаков М.А.,8.0
